## Mission to Mars

### Nasa Mars News

In [72]:
#import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pandas as pd
from splinter import Browser

In [73]:
#NASA Mars News URL to be scraped for latest news and paragraph title
URL = "https://mars.nasa.gov/news/"

# Retrieve page with the requests module
response = requests.get(URL)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [74]:
#Retrieve the latest news title from Nasa Mars News
news_title = soup.find('div', class_='content_title').text

#Retrieve the latest news paragraph from Nasa Mars News
news_para = soup.find('div', class_='rollover_description_inner').text

print(f'News Title:{news_title}')
print(f'News Paragraph:{news_para}')

News Title:

Opportunity Hunkers Down During Dust Storm


News Paragraph:
It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or even months, before skies are clear enough for NASA's Opportunity rover to recharge its batteries and phone home. 



### JPL Mars Space Images - Featured Image

In [75]:
# JPL url to scrape featured images
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [76]:
# visit JPL url
browser.visit(jpl_url)

In [77]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
jpl_soup = bs(html, 'html.parser')

In [78]:
# Retrieve all elements that contain url information
result = jpl_soup.find('footer')

#run the loop to find the image url
a = result.find('a')
featured_image_url = result.a['data-fancybox-href']
print('https://www.jpl.nasa.gov' + featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17171_ip.jpg


### Mars Weather

In [79]:
#mars weather URL
weather_url = 'https://twitter.com/marswxreport?lang=en'
# visit weather url
browser.visit(weather_url)

In [80]:
# HTML object
weather_html = browser.html
# Parse HTML with Beautiful Soup
weather_soup = bs(weather_html, 'html.parser')


In [81]:
#get mars weather latest tweet from website
mars_weather = weather_soup.find('p', class_='TweetTextSize').text
print(mars_weather)

Wondering where the Mars weather reports have been?  Curiosity is back online after a memory anomaly on Sol 2172. Read more here: https://mars.jpl.nasa.gov/msl/mission/mars-rover-curiosity-mission-updates/?mu=sol-2204-curiosity-science-is-baaaack …


### Mars Facts

In [82]:
#Mars facts URL
facts_url = 'http://space-facts.com/mars/'

#use pandas to scrape the table
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [83]:
#use pandas to convert data into a table
df = tables[0]
df.columns = ['0', '1']
df.rename(columns={'0': 'Description', '1': 'Values'})

,Description,Values
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [84]:
#Use Pandas to convert the data to a HTML table string.
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    

### Mars Hemispheres

In [95]:
# use the USGS Astrogeology site
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# setting up splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(usgs_url)

In [96]:
# HTML object
usgs_html = browser.html
# Parse HTML with Beautiful Soup
usgs_soup = bs(usgs_html, 'html.parser')

In [117]:
# Create an empty list to hold dictionaries of hemisphere title with the image url string
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css("a.product-item")['href']

# Loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item")['href'][i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    #sample_elem = browser.find_link_by_text('Sample').first
    #hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    #hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    #hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    #browser.back()

AttributeError: 'str' object has no attribute 'click'

In [111]:
#a = usgs_soup.find("img", class_="wide-image")["src"]

a = browser.find_by_css('img[class="wide-image"]')
for i in a:
    k = i['scr']

In [106]:
#a = usgs_soup.find_all('div', class_='wide-image-wrapper')

#for i in a:
    #cerberus_url = i.find("img", class_="wide-image")["src"]
    #cerberus_title = i.find("h2",class_="title").text
    #print(cerberus_title)
    

#k = []

cerberus_url = usgs_soup.find('div', 'downloads').a['href']

AttributeError: 'NoneType' object has no attribute 'a'

In [107]:
# create a empty list to hold the dictionary for hemisphere title and image url
#hemisphere_url = []
href = []
# Get a list of all of the hemispheres
links = browser.find_by_css('div[class="description"] a')

# Loop through those links, click the link, find the sample anchor, return the href
for i in links:
    #href.append(i['href'])
    #hemisphere = {}
    k = i['href']
    href.append(k)
    #print(i['href'])
    # We have to find the elements on each loop to avoid a stale element exception
    
    
    # Next, we find the Sample image anchor tag and extract the href
    #sample_elem = browser.find_link_by_text('Sample')
    #hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    #hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    #hemisphere_url.append(hemisphere)
    
    # Finally, we navigate backwards
    #browser.back()

href

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [110]:
for h in range(len(href)):
    #hemisphere = {}
    a = browser.find_by_css("img", class_="wide-image")["src"]
    #print(a['href'])
    #sample_elem = browser.find_link_by_text('Sample')[h]
    #hemisphere['img_url'] = sample_elem(h['href'])
    #print(sample_elem['href'])

TypeError: find_by_css() got an unexpected keyword argument 'class_'

In [66]:
for h in href:
    o = h.find("img", class_="wide-image")["src"]
    

TypeError: find() takes no keyword arguments